<h1> Segmenting and Clustering Neighbourhoods in Toronto </h1>
<i> Submission by Sebastiaan Hemmer on Oct, 13th 2019 </i>


<h2> Part I: Scraping neighbourhood data </h2>

In [1]:
# First we import all necessary libraries
import requests
import pandas as pd
import numpy as np
import folium

#!pip install beautifulsoup4
#!pip install html5lib
#!pip install lxml

from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

Now lets scrape the wiki contents using requests and parse using BeautifulSoup.

In [2]:
wiki = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
res = requests.get(wiki).text
soup = BeautifulSoup(res,'html')

Now that we have parsed the webpage, let's find the table on the wiki page and parse it into
a pandas dataframe.

In [3]:
tb = soup.find('table',{'class':'wikitable sortable'})
df = pd.read_html(tb.prettify())[0]
df.rename(columns = {'Postcode': 'PostalCode'}, inplace=True)
df.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [4]:
# Now remove all unassigned boroughs
df = df.loc[df['Borough'] != 'Not assigned']
# And group dataframe based on Postcode & Borough
df = df.groupby(['PostalCode', 'Borough']).agg(Neighbourhood=('Neighbourhood', ', '.join)).reset_index()
df.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Now let's fill unassigned neighbourhoods with the borough value.

In [5]:
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']
df.iloc[85]

PostalCode                M7A
Borough          Queen's Park
Neighbourhood    Queen's Park
Name: 85, dtype: object

And finally, print the shape of the dataframe.

In [6]:
df.shape

(103, 3)

<h2> Part II: Fetching Geocoding data </h2>

We couldnt get the Geocoder working so will use the provided .csv instead.

In [7]:
postal_codes = pd.read_csv('http://cocl.us/Geospatial_data')
postal_codes.rename(columns = {'Postal Code': 'PostalCode'}, inplace=True)
postal_codes.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Now let's join the Latitude and Longitude columns to our df dataframe.

In [8]:
df = df.merge(postal_codes, on='PostalCode', how='left')
print(df.shape)
df.head()

(103, 5)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


<h2> Part III: Clustering Neighbourhoods using Foursquare data </h2>

Define Foursquare Credentials and Version

In [56]:
CLIENT_ID = 'YOUR_ID'
CLIENT_SECRET = 'YOUR_SECRID'
VERSION = '20191013'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YOUR_ID
CLIENT_SECRET:YOUR_SECRID


<h4> Finding Neighbourhood venues </h4>
First we set the additional variables, we only want to find 100 venues in a radius of 500m

In [10]:
# set variables
limit = 100
radius = 500

We are only interested in the City of Toronto data so will exclude any boroughs not containing 'Toronto'.

In [11]:
df = df.loc[df['Borough'].str.contains('Toronto') == True]
df['Borough'].value_counts()

Downtown Toronto    18
Central Toronto      9
West Toronto         6
East Toronto         5
Name: Borough, dtype: int64

Now let's create a function to find the top nearby venues per Neighbourhood.

In [12]:
def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now let's fetch the top 100 venues for each neighbourhood in our dataframe and write to a new dataframe.

In [13]:
venues = getNearbyVenues(names=df['Neighbourhood'], latitudes=df['Latitude'], longitudes=df['Longitude'])
print(venues.shape)
venues.head()

(1724, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


Let's check how many venues were returned for each Neighbourhood.

In [14]:
venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,57,57,57,57,57,57
"Brockton, Exhibition Place, Parkdale Village",25,25,25,25,25,25
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",17,17,17,17,17,17
"Cabbagetown, St. James Town",44,44,44,44,44,44
Central Bay Street,88,88,88,88,88,88
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16


<h4> One hot encoding </h4>
Now let's use one hot encoding to split the categories into different columns.

In [15]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add Neighborhood column back to venues dataframe
onehot['Neighbourhood'] = venues['Neighbourhood'] 

# move Neighbourhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Next, let's group rows by neighbourhood and by taking the mean of the frequency of occurrence of each category.

In [16]:
grouped = onehot.groupby('Neighbourhood').mean().reset_index()
grouped.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.04,0.0,...,0.0,0.0,0.020000,0.0,0.0,0.0,0.01,0.0,0.01,0.000000
1,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,...,0.0,0.0,0.017544,0.0,0.0,0.0,0.00,0.0,0.00,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.00,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.00,0.058824
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.0,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.00,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.00,0.000000


<h4> Most popular categories </h4>
Now let's write a function to sort the venue categories by occurance.

In [17]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Next we create a new dataframe and display the top 10 venues for each neighbourhood.

In [49]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Neighbourhood'] = grouped['Neighbourhood']

for ind in np.arange(grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,American Restaurant,Bar,Thai Restaurant,Restaurant,Burger Joint,Sushi Restaurant,Hotel,Asian Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Bakery,Cheese Shop,Seafood Restaurant,Café,Farmers Market,Steakhouse,Italian Restaurant
2,"Brockton, Exhibition Place, Parkdale Village",Café,Coffee Shop,Breakfast Spot,Sandwich Place,Stadium,Caribbean Restaurant,Bar,Climbing Gym,Bakery,Furniture / Home Store
3,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Garden,Park,Pizza Place,Recording Studio,Restaurant,Burrito Place,Skate Park,Brewery,Smoke Shop
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Lounge,Airport Terminal,Boutique,Harbor / Marina,Sculpture Garden,Boat or Ferry,Bar,Plane,Coffee Shop


<h4> Clustering Neighbourhoods </h4>
We will use k-means clustering to cluster the neighbourhoods into different clusters

In [50]:
# set number of clusters
kclusters = 7

grouped_clustering = grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighbourhood.

In [51]:
# add clustering labels
venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_.astype(int))

merged = df # set copy of dataframe

# merge venue data with neighbourhood data, note that how=right means we only keep neighbourhoods
# that have venue data available.
merged = merged.join(venues_sorted.set_index('Neighbourhood'), on='Neighbourhood', how='right')

merged.head() # check the last columns!

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,6,Health Food Store,Neighborhood,Pub,Trail,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Brewery,Fruit & Vegetable Store,Juice Bar,Spa,Liquor Store
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Park,Sandwich Place,Pizza Place,Brewery,Burger Joint,Burrito Place,Pub,Coffee Shop,Pet Store,Movie Theater
43,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Italian Restaurant,Bakery,American Restaurant,Yoga Studio,Comfort Food Restaurant,Brewery,Seafood Restaurant,Sandwich Place
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,3,Park,Swim School,Bus Line,Yoga Studio,Doner Restaurant,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant


In [46]:
# Check if we have missed any clusters
merged[merged['Cluster Labels'].isna() == True]

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


<h4> Visualization </h4>
Finally, let's visualize the resulting clusters! First we start by defining the coordinates of
Toronto so we can center our map there.

In [47]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Now let's create our map and plot the neighbourhoods with different clusters as color code.

In [52]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['Latitude'], merged['Longitude'], merged['Neighbourhood'], merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [53]:
merged['Cluster Labels'].value_counts()

0    32
6     1
5     1
4     1
3     1
2     1
1     1
Name: Cluster Labels, dtype: int64

In [55]:
merged.loc[merged['Cluster Labels'] == 0, merged.columns[[1] + list(range(5, merged.shape[1]))]].head()

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,East Toronto,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Brewery,Fruit & Vegetable Store,Juice Bar,Spa,Liquor Store
42,East Toronto,0,Park,Sandwich Place,Pizza Place,Brewery,Burger Joint,Burrito Place,Pub,Coffee Shop,Pet Store,Movie Theater
43,East Toronto,0,Café,Coffee Shop,Italian Restaurant,Bakery,American Restaurant,Yoga Studio,Comfort Food Restaurant,Brewery,Seafood Restaurant,Sandwich Place
45,Central Toronto,0,Sandwich Place,Breakfast Spot,Gym,Hotel,Food & Drink Shop,Park,Dog Run,Clothing Store,Eastern European Restaurant,Doner Restaurant
46,Central Toronto,0,Sporting Goods Shop,Clothing Store,Coffee Shop,Yoga Studio,Chinese Restaurant,Spa,Mexican Restaurant,Burger Joint,Salon / Barbershop,Diner
